In [2]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

print("✅ Keys loaded:")
print(f"OpenAI API Key: {openai_api_key[:5]}***")
print(f"Tavily API Key: {tavily_api_key[:5]}***")

openai_client = OpenAI(api_key=openai_api_key)

✅ Keys loaded:
OpenAI API Key: sk-pr***
Tavily API Key: tvly-***


In [3]:
def print_markdown(text: str):
    display(Markdown(text))
    

In [4]:
import json  # Import the json module for handling JSON data
from typing_extensions import TypedDict  # Import TypedDict for type hinting
from agents import function_tool

In [5]:
# Define a TypedDict for the expected parameters for the Tavily search function
# A TypedDict is like a blueprint for a dictionary in Python
# It tells Python exactly what keys the dictionary should have and what type of values go with each key.

class TavilySearchParams(TypedDict):
    query: str         # The search query string
    max_results: int   # The maximum number of results to return

In [11]:
# Let's define a function that searches the web using the Tavily API and gives back a short summary of the top results.
# Decorate the function as a FunctionTool for OpenAI Agents SDK
@function_tool
def tavily_search(params: TavilySearchParams) -> str:
    """
    Calls the Tavily API and returns a string summary of top search results.

    Args:
        params (TavilySearchParams): Dictionary with 'query' (str) and 'max_results' (int).

    Returns:
        str: A formatted string summarizing the top search results, or an error message.
    """
    print("Start tavily_search")
    
    # The web address (endpoint) for sending search requests to Tavily ( Tavily API endpoint)
    url = "https://api.tavily.com/search" 

    # Tell the API that we’re sending JSON data
    headers = {"Content-Type": "application/json"}  

    # What we’re sending to the API:
    # Our secret API key (so Tavily knows it's us)
    # The search text (query)
    # How many results we want (defaults to 2 if not given)
    payload = {
        "api_key": tavily_api_key,  # Use the Tavily API key from environment
        "query": params["query"],   # The search query from params
        "max_results": params.get("max_results", 2),  # Use max_results from params, default to 2 if not provided
    }

    # Send the search request to Tavily (POST means we’re sending data)
    response = requests.post(url, json = payload, headers = headers) 

    # Check if the search worked (200 = OK)
    if response.status_code == 200:  # If the request was successful
        results = response.json().get("results", [])  # Extract the 'results' list from the response JSON
        
        # Build a summary string with each result's title and content, numbered
        summary = "\n".join([f"{i+1}. {r['title']}: {r['content']}" for i, r in enumerate(results)])
        return summary if summary else "No relevant results found."  # Return summary or fallback message
    else:
        return f"Tavily API error: {response.status_code}"  # Return error message with status code if request failed

In [12]:
# Let's add memory to our agent
from agents import SQLiteSession
session = SQLiteSession("live_researcher_practice")

In [13]:
from agents import Agent

live_researcher_agent = Agent(name = "Live Market Researcher",
                             instructions = """
CONTEXT:
You are a world-class market research assistant with access to real-time web search via the tavily_search tool.

INSTRUCTION:
- Analyze the user's question and determine if recent or real-time information is needed.
- If the question involves recent events, news, or product info, always call tavily_search.
- Summarize search results clearly and concisely, do not copy-paste.
- Always start your answer with: "🔍 According to a web search …"

INPUT:
You will receive a conversation history and the latest user question. Use the full context to inform your response.

OUTPUT:
Provide a clear, well-structured answer that references the search results when appropriate. If you use tavily_search, integrate the findings into your summary.
""",
    model = "gpt-4.1-mini",
    tools = [tavily_search])

print("✅ Agent created with Tavily tool.")

✅ Agent created with Tavily tool.


In [14]:
from agents import Runner

# First question
q1 = "What are people saying about the new GPT-5 Model?"

print_markdown(f"**User:** {q1}")

run1 = await Runner.run(
    starting_agent = live_researcher_agent,
    input = q1,
    session = session,
)

print_markdown(f"### 🤖 Agent’s Answer\n{run1.final_output}")



**User:** What are people saying about the new GPT-5 Model?

Start tavily_search


### 🤖 Agent’s Answer
🔍 According to a web search, the GPT-5 model by OpenAI has received a mixed but generally positive reception with several key highlights:

- GPT-5 is recognized for being a strong model in coding, reasoning, and agentic tasks, with configurable reasoning effort and superior tool-use skills. It is positioned as the best model globally for coding, which is particularly important for advancing future models like GPT-6.
- The model is considered fast, cost-effective, and available to most users, dominating the price-performance space compared to others.
- OpenAI has worked on making GPT-5 more customizable and "warmer" in personality, acknowledging that no single model fits all user preferences.
- However, some users find the upgrade underwhelming compared to competitors like Anthropic’s Claude 4/4.1 models, and GPT-5 still struggles with creative writing and some advanced reasoning challenges like ARC-AGI 2.
- OpenAI provides different versions like GPT-5.2, GPT-5 mini, and GPT-5 nano to balance performance, speed, and cost efficiency.
- There is ongoing emphasis on safety features and steerability to prevent misuse and improve user experience.

In summary, GPT-5 is noted for its strengths in coding and efficiency, with improvements in user customization and personality, but some users feel certain aspects remain a challenge compared to the latest models by competitors.